# Necessities

In [70]:
# ------------------------------------------------------------------
# Imports
# ------------------------------------------------------------------
# Basic data processing libraries
import pandas as pd
import numpy as np
import os
import torch

# Graph data processing libraries
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# Libraries for (G)NNs
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import torch.nn as nn
from sklearn.metrics import roc_auc_score, f1_score

# ------------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------------
def show_df_info(df):
    print(df.info())
    print('####### Repeat ####### \n', df.duplicated().any())
    print('####### Count ####### \n', df.nunique())
    print('####### Example ####### \n',df.head())

def label_statics(label_df, label_list):
    print("####### nCount #######")
    for label in label_list:
        print(label_df[label].value_counts())
    print("####### nPercent #######")
    for label in label_list:
        print(label_df[label].value_counts()/label_df.shape[0])

# ------------------------------------------------------------------
# Data stuff
# ------------------------------------------------------------------
base_path = os.getcwd()
input_ali_data_path = base_path

# Load the data files
user_labels_path = os.path.join(input_ali_data_path, "nba.csv")
user_edges_path = os.path.join(input_ali_data_path, "nba_relationship.csv")

# Create dataframes to store the information from the .csv files
user_labels = pd.read_csv(user_labels_path)
user_edges = pd.read_csv(user_edges_path)

# Prepare the data for GNNs
node_features = torch.tensor(user_labels.iloc[:, 1:].values, dtype=torch.float)
edge_index = torch.tensor(user_edges.values, dtype=torch.long).t().contiguous()

user_edges = user_edges[user_edges['uid1'].isin(user_labels['user_id']) & user_edges['uid2'].isin(user_labels['user_id'])]

# Extract node features from user_labels dataframe
node_features = user_labels.iloc[:, 1:] 
node_features = torch.tensor(node_features.values, dtype=torch.float)

# Extract edges from user_edges dataframe
edges = user_edges[['uid1', 'uid2']]
edges['uid1'] = edges['uid1'].map(dict(zip(user_labels['user_id'], range(len(user_labels)))))
edges['uid2'] = edges['uid2'].map(dict(zip(user_labels['user_id'], range(len(user_labels)))))

# Convert edges dataframe to tensor
edges_tensor = torch.tensor(edges.values, dtype=torch.long).t().contiguous()

# Create edge_index tensor
edge_index = edges_tensor

user_labels['SALARY'] = user_labels['SALARY'].map({-1: 0, 0: 1, 1: 1})

# Create torch-geometric data
data = Data(x=node_features, edge_index=edge_index)

num_nodes = node_features.size(0)
num_classes = 2 
num_node_features = data.num_node_features

# Create masks for training, and testing
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)

# 60-20-20 Train and Test data split
num_train = int(num_nodes * 0.6)
num_val = int(num_nodes * 0.8)
train_mask[:num_train] = True
val_mask[num_train:num_val] = True
test_mask[num_val:] = True

data.train_mask = train_mask
data.test_mask = test_mask
data.val_mask = val_mask

# Labels from the data (in this case: Job Classification)
data.y = torch.tensor(user_labels['SALARY'].values, dtype=torch.long)


# ------------------------------------------------------------------
# Set Device
# ------------------------------------------------------------------

def set_device():
    return torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# ------------------------------------------------------------------
# Loss
# ------------------------------------------------------------------

def fairness_aware_loss(output, data, sensitive_attr, alpha=0, beta=0, gamma=0, delta=0):
    target = data.y[data.train_mask]
    # standard_loss = F.cross_entropy(output, target)
    standard_loss = F.nll_loss(output, target)

    labels = data.y[train_mask]
    pos_prob = torch.exp(output[:, 1])
    neg_prob = torch.exp(output[:, 0])
    # pos_prob = torch.sigmoid(output[:, 1])
    # neg_prob = 1 - pos_prob
    predictions = output.argmax(dim=1)

    # Statistical Parity Regularization
    sp_reg = torch.abs(pos_prob[sensitive_attr == 1].mean() - pos_prob[sensitive_attr == 0].mean())

    # # Calculating FPR and TPR for each group
    # fpr_group1 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 1)).float().mean()
    # fpr_group0 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 0)).float().mean()
    # tpr_group1 = ((predictions == 1) & (labels == 1) & (sensitive_attr == 1)).float().mean()
    # tpr_group0 = ((predictions == 1) & (labels == 1) & (sensitive_attr == 0)).float().mean()

    # Treatment Equality Regularization
    fp_diff = (neg_prob * (labels == 0) * (sensitive_attr == 1)).float().mean() - \
              (neg_prob * (labels == 0) * (sensitive_attr == 0)).float().mean()
    fn_diff = (pos_prob * (labels == 1) * (sensitive_attr == 1)).float().mean() - \
              (pos_prob * (labels == 1) * (sensitive_attr == 0)).float().mean()
    treatment_reg = torch.abs(fp_diff) + torch.abs(fn_diff)
    # treatment_reg = torch.abs(fn_diff)

    # fn_group_1 = ((predictions == 0) & (labels == 1) & (sensitive_attr == 1)).sum()
    # fp_group_1 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 1)).sum()

    # fn_group_0 = ((predictions == 0) & (labels == 1) & (sensitive_attr == 0)).sum()
    # fp_group_0 = ((predictions == 1) & (labels == 0) & (sensitive_attr == 0)).sum()
    
    # ratio_group_1 = fn_group_1 / fp_group_1 if fp_group_1 != 0 else torch.tensor(float('inf'))
    # ratio_group_0 = fn_group_0 / fp_group_0 if fp_group_0 != 0 else torch.tensor(float('inf'))
    # treatment_reg = torch.abs(ratio_group_1 - ratio_group_0)

    # Equal Opportunity Difference Regularization
    eod_reg = torch.abs((pos_prob * (labels == 1) * (sensitive_attr == 1)).float().mean() - \
                        (pos_prob * (labels == 1) * (sensitive_attr == 0)).float().mean())

    # Overall Accuracy Equality Difference Regularization
    oaed_reg = torch.abs((pos_prob * (sensitive_attr == 1)).float().mean() - \
                         (pos_prob * (sensitive_attr == 0)).float().mean())

    penalty = alpha + beta + gamma + delta
    
    # Combine losses
    combined_loss = (1-penalty)*standard_loss
    + alpha * sp_reg
    + beta * treatment_reg
    + gamma * eod_reg
    + delta * oaed_reg
    
    return combined_loss

# ------------------------------------------------------------------
# Fairness Metrics
# ------------------------------------------------------------------

def calculate_fairness(label, predictions, sens_attr='Gender', balanced=False):
    """
    Calculate various fairness metrics.

    Args:
    label: Actual labels (binary).
    predictions: Model predictions (binary).
    sens_attr: Binary sensitive attribute for fairness evaluation.

    Returns:
    A dictionary containing SPD, EOD, OAED, and TED values.
    """
    if balanced is False:
        labels = torch.tensor(user_labels[label].values, dtype=torch.long)
        sensitive_attribute = torch.tensor(user_labels[sens_attr].values, dtype=torch.long)
    else:
        labels = torch.tensor(filtered_user_labels[label].values, dtype=torch.long)
        sensitive_attribute = torch.tensor(filtered_user_labels[sens_attr].values, dtype=torch.long)
    
    labels = labels.to(set_device())
    sensitive_attribute = sensitive_attribute.to(set_device())

    predictions = predictions.float()
    labels = labels.float()
    sensitive_attribute = sensitive_attribute.float()

    def statistical_parity_difference():
        prob_group_1 = predictions[sensitive_attribute == 1].mean()
        prob_group_0 = predictions[sensitive_attribute == 0].mean()
        return abs(prob_group_1 - prob_group_0), prob_group_0, prob_group_1

    def equal_opportunity_difference():
        tpr_group_1 = predictions[(labels == 1) & (sensitive_attribute == 1)].mean()
        tpr_group_0 = predictions[(labels == 1) & (sensitive_attribute == 0)].mean()
        return abs(tpr_group_1 - tpr_group_0), tpr_group_0, tpr_group_1

    def overall_accuracy_equality_difference():
        acc_group_1 = (predictions[sensitive_attribute == 1] == labels[sensitive_attribute == 1]).float().mean()
        acc_group_0 = (predictions[sensitive_attribute == 0] == labels[sensitive_attribute == 0]).float().mean()
        return abs(acc_group_1 - acc_group_0), acc_group_0, acc_group_1

    def treatment_equality_difference():
        fn_group_1 = ((predictions == 0) & (labels == 1) & (sensitive_attribute == 1)).sum()
        fp_group_1 = ((predictions == 1) & (labels == 0) & (sensitive_attribute == 1)).sum()

        fn_group_0 = ((predictions == 0) & (labels == 1) & (sensitive_attribute == 0)).sum()
        fp_group_0 = ((predictions == 1) & (labels == 0) & (sensitive_attribute == 0)).sum()

        ratio_group_1 = fn_group_1 / fp_group_1 if fp_group_1 != 0 else float('inf')
        ratio_group_0 = fn_group_0 / fp_group_0 if fp_group_0 != 0 else float('inf')

        return abs(ratio_group_1 - ratio_group_0), ratio_group_0, ratio_group_1, fn_group_1, fp_group_1, fn_group_0, fp_group_0

    # Calculating each fairness metric
    spd, sp_g0, sp_g1 = statistical_parity_difference()
    eod, eod_g0, eod_g1 = equal_opportunity_difference()
    oaed, oaed_g0, oaed_g1 = overall_accuracy_equality_difference()
    ted, ted_g0, ted_g1, fn_group_1, fp_group_1, fn_group_0, fp_group_0 = treatment_equality_difference()

    return {
        'Statistical Parity Difference': spd,
        'Statistical Parity Group with S=0': sp_g0,
        'Statistical Parity Group S=1': sp_g1,
        'Equal Opportunity Difference': eod,
        'Equal Opportunity Group with S=0': eod_g0,
        'Equal Opportunity Group S=1': eod_g1,
        'Overall Accuracy Equality Difference': oaed,
        'Overall Accuracy Group with S=0': oaed_g0,
        'Overall Accuracy Group S=1': oaed_g1,
        'Treatment Equality Difference': ted,
        'Treatment Equality Group with S=0': ted_g0,
        'Treatment Equality Group S=1': ted_g1
        # 'False Negatives Group 1': fn_group_1,
        # 'False Positives Group 1': fp_group_1,
        # 'False Negatives Group 0': fn_group_0,
        # 'False Positives Group 0': fp_group_0
    }

# ------------------------------------------------------------------
# Model Training
# ------------------------------------------------------------------

# Train the model
def training(model, data, optimizer, epochs=1000, fairness=False, alpha=0, beta=0, gamma=0, delta=0):
    model.to(set_device())
    data.to(set_device())
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        
        if fairness:
            loss = fairness_aware_loss(out[data.train_mask], data, data.x[data.train_mask, -1],
                                       alpha=alpha, beta=beta, gamma=gamma, delta=delta)
            
        else:
            # criterion = torch.nn.CrossEntropyLoss()
            # criterion = torch.nn.BCELoss()
            criterion = torch.nn.NLLLoss()
            loss = criterion(out[data.train_mask], data.y[data.train_mask])

        loss.backward()
        optimizer.step()

        metrics = test(model, data)

        if epoch % 10 == 0:
            print(f'Epoch {epoch} | Loss: {loss.item()} | \n AUC_ROC: {metrics["AUC_ROC"]} | F1 Score: {metrics["F1_Score"]} | SPD: {metrics["parity"]} | EOD: {metrics["equality"]}')

# ------------------------------------------------------------------
# Model Testing
# ------------------------------------------------------------------

# Test the model
def test(model, data, balanced=False):
    # model.to('cpu')
    # data.to('cpu')
    model.to(set_device())
    data.to(set_device())
    
    model.eval()
    with torch.inference_mode():
      out = model(data.x, data.edge_index)

    _, pred = model(data.x, data.edge_index).max(dim=1)
    correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    accuracy = correct / int(data.test_mask.sum())
    # print(f'Accuracy: {accuracy}')

    # Convert model outputs to binary predictions
    predictions = out.argmax(dim=1)

    fairness_metrics = calculate_fairness(label='GoodCustomer', predictions=predictions, sens_attr='Gender', balanced=balanced)
    fairness_metrics['Accuracy'] = accuracy

    return fairness_metrics

# ------------------------------------------------------------------
# Print Metrics
# ------------------------------------------------------------------

# def print_metrics(metrics):
#     for key, value in metrics.items():
#         print(f"\n{key} : {value:.5f}")

def print_metrics(metrics):
    count = -1

    for key, value in metrics.items():
        count += 1
        if count == 3:
            print(f"\n\n{key} : {value:.5f}")
            count = 0
        else:
            print(f"{key} : {value:.5f}")

In [71]:
def test(model, data, val=True, balanced=False):
    model.to(set_device())
    data.to(set_device())
    
    if val==True:
      mask = data.val_mask
    else:
      mask = data.test_mask

    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        predictions = out.argmax(dim=1)

    # Compute accuracy
    correct = int(predictions[mask].eq(data.y[mask]).sum().item())
    accuracy = correct / int(mask.sum())
    
    # Extract the predictions and the true labels
    y_true = data.y[mask].cpu().numpy()
    y_pred = predictions[mask].cpu().numpy()
    
    # Compute F1 score
    f1 = f1_score(y_true, y_pred, average='binary')

    # Compute AUC-ROC score
    y_probs = out[mask][:, 1].cpu().numpy() 
    auc_roc = roc_auc_score(y_true, y_probs)
    
    fairness_metrics = fair_metric('SALARY', predictions, 'country')
    fairness_metrics['Accuracy'] = accuracy
    fairness_metrics['F1_Score'] = f1
    fairness_metrics['AUC_ROC'] = auc_roc

    return fairness_metrics

def fair_metric(labels, pred, sens):
	
	labels = user_labels[labels].values
	sens = user_labels[sens].values
	
	idx_s0 = sens==0
	idx_s1 = sens==1

	idx_s0_y1 = np.bitwise_and(idx_s0, labels==1)
	idx_s1_y1 = np.bitwise_and(idx_s1, labels==1)

	parity = abs(sum(pred[idx_s0])/sum(idx_s0)-sum(pred[idx_s1])/sum(idx_s1))
	equality = abs(sum(pred[idx_s0_y1])/sum(idx_s0_y1)-sum(pred[idx_s1_y1])/sum(idx_s1_y1))
    
	return {"parity": parity.item(), "equality": equality.item()}

In [72]:
sens_attribute_tensor = torch.tensor(user_labels['country'].values, dtype=torch.long)
sens_attribute_tensor = sens_attribute_tensor.to(set_device())

# GCN Model

In [73]:
class GCN(nn.Module):
	def __init__(self, nfeat, nhid=128, nclass=2, dropout=0):
		super(GCN, self).__init__()
		self.body = GCN_Body(nfeat,nhid,dropout)
		self.fc = nn.Linear(nhid, nclass)

		for m in self.modules():
			self.weights_init(m)

	def weights_init(self, m):
		if isinstance(m, nn.Linear):
			torch.nn.init.xavier_uniform_(m.weight.data)
			if m.bias is not None:
				m.bias.data.fill_(0.0)

	def forward(self, x, edge_index):
		x = self.body(x, edge_index)
		x = self.fc(x)
		return F.log_softmax(x, dim=1)
		# return x

In [74]:
class GCN_Body(nn.Module):
	def __init__(self, nfeat, nhid, dropout):
		super(GCN_Body, self).__init__()
		self.gc1 = GCNConv(nfeat, nhid)

	def forward(self, x, edge_index):
		x = self.gc1(x, edge_index)
		return x

In [75]:
gcn_model = GCN(data.num_node_features, nhid=128, nclass=2)
optimizer_gcn_model = torch.optim.Adam(gcn_model.parameters(), lr=1e-4, weight_decay=1e-5)

In [76]:
training(model=gcn_model, 
         data=data, 
         optimizer=optimizer_gcn_model, 
         fairness=False,  
         epochs=1500)

Epoch 0 | Loss: 6.057296276092529 | 
 AUC_ROC: 0.519400352733686 | F1 Score: 0.8750000000000001 | SPD: 5.960464477539063e-08 | EOD: 0.0
Epoch 10 | Loss: 3.1994152069091797 | 
 AUC_ROC: 0.503968253968254 | F1 Score: 0.8750000000000001 | SPD: 0.009345829486846924 | EOD: 0.012048184871673584
Epoch 20 | Loss: 1.2042274475097656 | 
 AUC_ROC: 0.3509700176366843 | F1 Score: 0.4086021505376344 | SPD: 0.05929526686668396 | EOD: 0.010267138481140137
Epoch 30 | Loss: 1.1087586879730225 | 
 AUC_ROC: 0.3880070546737213 | F1 Score: 0.8591549295774649 | SPD: 0.011145532131195068 | EOD: 0.014405429363250732
Epoch 40 | Loss: 0.9991555213928223 | 
 AUC_ROC: 0.3615520282186948 | F1 Score: 0.8208955223880596 | SPD: 0.038298726081848145 | EOD: 0.026348888874053955
Epoch 50 | Loss: 0.9658806324005127 | 
 AUC_ROC: 0.36155202821869487 | F1 Score: 0.832116788321168 | SPD: 0.03334170579910278 | EOD: 0.01665794849395752
Epoch 60 | Loss: 0.9418555498123169 | 
 AUC_ROC: 0.3597883597883597 | F1 Score: 0.82089552238

In [77]:
print("Here are the values for the GCN model")

metrics_gcn_model = test(gcn_model, data)

print_metrics(metrics_gcn_model)

Here are the values for the GCN model
parity : 0.01576
equality : 0.01069
Accuracy : 0.80247


F1_Score : 0.88235
AUC_ROC : 0.71340


# FAME

In [78]:
class FairnessAwareMessagePassingLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(FairnessAwareMessagePassingLayer, self).__init__(aggr='mean')  
        self.lin = nn.Linear(in_channels, out_channels)
        self.sensitive_attr = sens_attribute_tensor
        self.bias_correction = nn.Parameter(torch.rand(1))

    def forward(self, x, edge_index):        
        # Add self-loops 
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        x = self.lin(x)

        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)
    
    def message(self, x_j, edge_index, size):
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        group_difference = self.sensitive_attr[row] - self.sensitive_attr[col]
        
        # Adjust messages based on statistical parity
        fairness_adjustment = (1 + self.bias_correction * group_difference.view(-1, 1))

        return fairness_adjustment * norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        return aggr_out

In [79]:
class FairMP_GCN(torch.nn.Module):
    def __init__(self, data, layers=1, hidden=128, dropout=0):
        super(FairMP_GCN, self).__init__()
        self.conv1 = FairnessAwareMessagePassingLayer(data.num_node_features, hidden)
        self.convs = torch.nn.ModuleList()
        
        for i in range(layers - 1):
            self.convs.append(FairnessAwareMessagePassingLayer(hidden, hidden))
        
        # self.conv2 = FairnessAwareMessagePassingLayer(hidden, 2)
        self.fc = nn.Linear(hidden, 2)
        self.dropout = dropout

    def forward(self, x, edge_index, *args, **kwargs):
        x = F.relu(self.conv1(x, edge_index))
        # x = F.dropout(x, p=self.dropout, training=self.training)

        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
            x = F.dropout(x, p=self.dropout, training=self.training)

        # x = self.conv2(x, edge_index)
        x = self.fc(x)
        
        return F.log_softmax(x, dim=1)

In [80]:
Fair_gcn_model = FairMP_GCN(data, hidden=128)
optimizer_Fair_gcn_model = torch.optim.Adam(Fair_gcn_model.parameters(), lr=1e-4, weight_decay=1e-5)

In [81]:
training(model=Fair_gcn_model, 
         data=data, 
         optimizer=optimizer_Fair_gcn_model, 
         fairness=False,  
         epochs=1500)

Epoch 0 | Loss: 0.7694790363311768 | 
 AUC_ROC: 0.49735449735449727 | F1 Score: 0.0 | SPD: 0.0 | EOD: 0.0
Epoch 10 | Loss: 0.7106773257255554 | 
 AUC_ROC: 0.3245149911816578 | F1 Score: 0.08571428571428572 | SPD: 0.00293634831905365 | EOD: 0.003981143236160278
Epoch 20 | Loss: 0.6795920133590698 | 
 AUC_ROC: 0.4153439153439153 | F1 Score: 0.8148148148148149 | SPD: 0.34241610765457153 | EOD: 0.35709792375564575
Epoch 30 | Loss: 0.6645441055297852 | 
 AUC_ROC: 0.427689594356261 | F1 Score: 0.842857142857143 | SPD: 0.1495327353477478 | EOD: 0.15662646293640137
Epoch 40 | Loss: 0.6544420719146729 | 
 AUC_ROC: 0.4373897707231041 | F1 Score: 0.8671328671328671 | SPD: 0.06542056798934937 | EOD: 0.048192739486694336
Epoch 50 | Loss: 0.6466317772865295 | 
 AUC_ROC: 0.4391534391534391 | F1 Score: 0.8671328671328671 | SPD: 0.04672902822494507 | EOD: 0.03614455461502075
Epoch 60 | Loss: 0.6402316093444824 | 
 AUC_ROC: 0.4444444444444444 | F1 Score: 0.8750000000000001 | SPD: 5.960464477539063e-08 |

In [82]:
print("Here are the values for the GCN model")

metrics_Fair_gcn_model = test(Fair_gcn_model, data)

print_metrics(metrics_Fair_gcn_model)

Here are the values for the GCN model
parity : 0.00202
equality : 0.00634
Accuracy : 0.80247


F1_Score : 0.88406
AUC_ROC : 0.69489


# A-FAME

In [83]:
import torch
from torch_geometric.nn import MessagePassing
from torch.nn import Linear, Parameter
from torch_geometric.utils import add_self_loops, softmax
import torch.nn.functional as F

class Attention_FairMessagePassing(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(Attention_FairMessagePassing, self).__init__(aggr='add') 
        self.lin = Linear(in_channels, out_channels) 
        self.att = Linear(2 * out_channels, 1) 
        
        self.sensitive_attr = sens_attribute_tensor 
        self.bias_correction = Parameter(torch.rand(1))  

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        x = self.lin(x)

        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, edge_index, x_i, x_j, size_i):
        x_cat = torch.cat([x_i, x_j], dim=-1)  
        alpha = self.att(x_cat)

        row, col = edge_index
        group_difference = self.sensitive_attr[row] - self.sensitive_attr[col]

        fairness_adjustment = self.bias_correction * group_difference.view(-1, 1)
        alpha = alpha + fairness_adjustment

        alpha = softmax(alpha, edge_index[0], num_nodes=size_i)

        return alpha * x_j

    def update(self, aggr_out):
        return aggr_out
    
# GCN class that takes in the data as an input for dimensions of the convolutions
class Fair_Attention_MP_GCN(torch.nn.Module):
    def __init__(self, data, layers=1, hidden=128, dropout=0):
        super(Fair_Attention_MP_GCN, self).__init__()
        self.conv1 = Attention_FairMessagePassing(data.num_node_features, hidden)
        self.convs = torch.nn.ModuleList()
        
        for i in range(layers - 1):
            self.convs.append(Attention_FairMessagePassing(hidden, hidden))
        
        # self.conv2 = Attention_FairMessagePassing(hidden, 2)
        self.fc = Linear(hidden, 2)
        self.dropout = dropout

    def forward(self, x, edge_index, *args, **kwargs):
        x = F.relu(self.conv1(x, edge_index))
        # x = F.dropout(x, p=self.dropout, training=self.training)

        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
            # x = F.dropout(x, p=self.dropout, training=self.training)

        # x = self.conv2(x, edge_index)
        x = self.fc(x)
        
        return F.log_softmax(x, dim=1)

In [91]:
Fair_gat_model = Fair_Attention_MP_GCN(data, hidden=128)
optimizer_Fair_gat_model = torch.optim.Adam(Fair_gat_model.parameters(), lr=1e-4, weight_decay=1e-5)

training(model=Fair_gat_model, 
         data=data, 
         optimizer=optimizer_Fair_gat_model, 
         fairness=False,  
         epochs=1000)

Epoch 0 | Loss: 2.5903937816619873 | 
 AUC_ROC: 0.6058201058201058 | F1 Score: 0.8750000000000001 | SPD: 5.960464477539063e-08 | EOD: 0.0
Epoch 10 | Loss: 1.5143588781356812 | 
 AUC_ROC: 0.6437389770723104 | F1 Score: 0.8750000000000001 | SPD: 5.960464477539063e-08 | EOD: 0.0
Epoch 20 | Loss: 0.9135501980781555 | 
 AUC_ROC: 0.6613756613756614 | F1 Score: 0.8750000000000001 | SPD: 5.960464477539063e-08 | EOD: 0.0
Epoch 30 | Loss: 0.6360685229301453 | 
 AUC_ROC: 0.6679894179894179 | F1 Score: 0.8750000000000001 | SPD: 0.00337827205657959 | EOD: 0.0
Epoch 40 | Loss: 0.5220937728881836 | 
 AUC_ROC: 0.6604938271604939 | F1 Score: 0.881118881118881 | SPD: 0.008556485176086426 | EOD: 0.003352522850036621
Epoch 50 | Loss: 0.4998898506164551 | 
 AUC_ROC: 0.6578483245149912 | F1 Score: 0.881118881118881 | SPD: 0.035804569721221924 | EOD: 0.0187532901763916
Epoch 60 | Loss: 0.4970947802066803 | 
 AUC_ROC: 0.656084656084656 | F1 Score: 0.881118881118881 | SPD: 0.035804569721221924 | EOD: 0.0187532

In [92]:
print("Here are the values for the FairGAT model")

metrics_Fair_gat_model = test(Fair_gat_model, data)

print_metrics(metrics_Fair_gat_model)

Here are the values for the FairGAT model
parity : 0.00215
equality : 0.00136
Accuracy : 0.80247


F1_Score : 0.88571
AUC_ROC : 0.77601


# GAT Model

In [93]:
class GAT(nn.Module):
	def __init__(self, nfeat, nhid=128, nclass=2, dropout=0):
		super(GAT, self).__init__()
		self.body = GAT_Body(nfeat,nhid,dropout)
		self.fc = nn.Linear(nhid, nclass)

		for m in self.modules():
			self.weights_init(m)

	def weights_init(self, m):
		if isinstance(m, nn.Linear):
			torch.nn.init.xavier_uniform_(m.weight.data)
			if m.bias is not None:
				m.bias.data.fill_(0.0)

	def forward(self, x, edge_index):
		x = self.body(x, edge_index)
		x = self.fc(x)
		return F.log_softmax(x, dim=1)
		# return x

In [94]:
from torch_geometric.nn import GATConv

class GAT_Body(nn.Module):
	def __init__(self, nfeat, nhid, dropout):
		super(GAT_Body, self).__init__()
		self.gc1 = GATConv(nfeat, nhid)

	def forward(self, x, edge_index):
		x = self.gc1(x, edge_index)
		return x

In [95]:
gat_model = GAT(data.num_node_features, nhid=128)
optimizer_gat_model = torch.optim.Adam(gat_model.parameters(), lr=1e-4, weight_decay=1e-5)

training(model=gat_model, 
         data=data, 
         optimizer=optimizer_gat_model, 
         fairness=False,  
         epochs=1000)

Epoch 0 | Loss: 6.716121196746826 | 
 AUC_ROC: 0.5277777777777778 | F1 Score: 0.8750000000000001 | SPD: 5.960464477539063e-08 | EOD: 0.0
Epoch 10 | Loss: 3.2787342071533203 | 
 AUC_ROC: 0.45811287477954143 | F1 Score: 0.8750000000000001 | SPD: 5.960464477539063e-08 | EOD: 0.0
Epoch 20 | Loss: 1.6724714040756226 | 
 AUC_ROC: 0.45238095238095244 | F1 Score: 0.34883720930232553 | SPD: 0.0018312633037567139 | EOD: 0.00921948254108429
Epoch 30 | Loss: 1.187846302986145 | 
 AUC_ROC: 0.45149911816578486 | F1 Score: 0.881118881118881 | SPD: 0.018691599369049072 | EOD: 0.0
Epoch 40 | Loss: 0.9525163769721985 | 
 AUC_ROC: 0.4541446208112875 | F1 Score: 0.8 | SPD: 0.0020522475242614746 | EOD: 0.010214805603027344
Epoch 50 | Loss: 0.9152907729148865 | 
 AUC_ROC: 0.45943562610229277 | F1 Score: 0.8750000000000001 | SPD: 0.015313267707824707 | EOD: 0.00770038366317749
Epoch 60 | Loss: 0.8718482255935669 | 
 AUC_ROC: 0.45326278659611996 | F1 Score: 0.8382352941176471 | SPD: 0.05777972936630249 | EOD:

In [96]:
print("Here are the values for the GAT model")

metrics_gat_model = test(gat_model, data)

print_metrics(metrics_gat_model)

Here are the values for the GAT model
parity : 0.01553
equality : 0.00335
Accuracy : 0.80247


F1_Score : 0.88732
AUC_ROC : 0.61023


# Compare results

In [97]:
print(f"GCN model: {metrics_gcn_model}\n")
print(f"Fair GCN model: {metrics_Fair_gcn_model}\n")
print(f"GAT model: {metrics_gat_model}\n")
print(f"Fair GAT model: {metrics_Fair_gat_model}\n")

GCN model: {'parity': 0.015755295753479004, 'equality': 0.010686218738555908, 'Accuracy': 0.8024691358024691, 'F1_Score': 0.8823529411764706, 'AUC_ROC': 0.7134038800705468}

Fair GCN model: {'parity': 0.0020207762718200684, 'equality': 0.0063384175300598145, 'Accuracy': 0.8024691358024691, 'F1_Score': 0.8840579710144927, 'AUC_ROC': 0.6948853615520282}

GAT model: {'parity': 0.015534281730651855, 'equality': 0.003352522850036621, 'Accuracy': 0.8024691358024691, 'F1_Score': 0.8873239436619719, 'AUC_ROC': 0.6102292768959435}

Fair GAT model: {'parity': 0.0021469593048095703, 'equality': 0.0013619661331176758, 'Accuracy': 0.8024691358024691, 'F1_Score': 0.8857142857142857, 'AUC_ROC': 0.7760141093474426}



In [ ]:
Values for the FairGAT model - same model specifications as the one from Carlos,
but with one A-FAME instead of one GCNConv layer

parity : 0.00706
equality : 0.00207
Accuracy : 0.68000
F1_Score : 0.80952
AUC_ROC : 0.67496